# Project 13 - Cross validation for MNIST dataset

In [1]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets
from sklearn.model_selection import cross_val_score
import torchvision.transforms as transforms
from skorch import NeuralNetClassifier
torch.__version__

'2.3.1+cu121'

In [2]:
torch.manual_seed(123)

In [3]:
transform = transforms.ToTensor()
train = datasets.MNIST(root='Datasets/', train=True, download=True, transform=transform)

In [8]:
class torch_classifier(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3,3))
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(3,3))
        self.activation = nn.ReLU()
        self.bnorm = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(kernel_size=(2,2))
        self.flatten = nn.Flatten()

        self.linear1 = nn.Linear(in_features=32*5*5, out_features = 128)
        self.linear2 = nn.Linear(in_features=128, out_features = 10)
        self.output = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.2)

    def forward(self, X):
        X = X.view(-1, 1, 28, 28)
        X = self.pool(self.bnorm(self.activation(self.conv1(X))))
        X = self.pool(self.bnorm(self.activation(self.conv2(X))))
        X = self.flatten(X)

        X = self.dropout(self.activation(self.linear1(X)))
        x = self.dropout(self.activation(self.linear2(X)))
        X = self.output(X)

        return X
    
    


In [9]:
sklearn_classifier = NeuralNetClassifier(
    module = torch_classifier,
    criterion = torch.nn.CrossEntropyLoss,
    optimizer = torch.optim.Adam,
    max_epochs = 2,
    batch_size = 10,
    train_split=False
)

In [10]:
X = train.data.type('torch.FloatTensor')
y = train.targets.type('torch.LongTensor')

In [11]:
results = cross_val_score(sklearn_classifier, X, y, cv=5, scoring='accuracy')

  epoch    train_loss      dur
-------  ------------  -------
      1        0.2817  92.6140
      2        0.2240  84.6456
  epoch    train_loss      dur
-------  ------------  -------
      1        0.2676  86.2828
      2        0.2328  85.9289
  epoch    train_loss      dur
-------  ------------  -------
      1        0.2547  86.9708
      2        0.2257  89.0286
  epoch    train_loss      dur
-------  ------------  -------
      1        0.2546  87.2139
      2        0.2200  95.2089
  epoch    train_loss      dur
-------  ------------  -------
      1        0.2489  93.9175
      2        0.2186  94.1789


In [12]:
mean = results.mean()
std = results.std()

In [13]:
mean, std

(0.6400166666666667, 0.1441972934712869)

In [14]:
results

array([0.43808333, 0.49241667, 0.7515    , 0.77691667, 0.74116667])